# Making altair plots to save to Jeykll

## How to include plots from the Vega-lite editor

In [1]:
import pandas as pd
import numpy as np
import altair as alt
import os

In [2]:
building_url = "https://github.com/UIUC-iSchool-DataViz/is445_data/raw/main/building_inventory.csv"

In [3]:
buildings = pd.read_csv(building_url)
buildings.head()

,Agency Name,Location Name,Address,City,Zip code,County,Congress Dist,Congressional Full Name,Rep Dist,Rep Full Name,...,Bldg Status,Year Acquired,Year Constructed,Square Footage,Total Floors,Floors Above Grade,Floors Below Grade,Usage Description,Usage Description 2,Usage Description 3
0,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,1975,1975,144,1,1,0,Unusual,Unusual,Not provided
1,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
2,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
3,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
4,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided


In [4]:
buildings.loc[buildings["Year Constructed"]==0, "Year Constructed"] = np.nan
buildings.loc[buildings["Square Footage"]==0, "Square Footage"] = np.nan

In [5]:
stats = buildings.groupby("Year Constructed")["Square Footage"].describe()
stats.index = pd.to_datetime(stats.index.astype('int'), format='%Y')
stats.reset_index().head()

,Year Constructed,count,mean,std,min,25%,50%,75%,max
0,1753-01-01,6.0,2500.166667,1390.184220,558.0,1545.0,2727.5,3421.75,4184.0
1,1802-01-01,2.0,2220.000000,1943.129435,846.0,1533.0,2220.0,2907.00,3594.0
2,1810-01-01,3.0,1344.333333,1809.945948,216.0,300.5,385.0,1908.50,3432.0
3,1812-01-01,1.0,5500.000000,NaN,5500.0,5500.0,5500.0,5500.00,5500.0
4,1815-01-01,1.0,800.000000,NaN,800.0,800.0,800.0,800.00,800.0


In [6]:
stats_melt = stats.reset_index().melt("Year Constructed", var_name="Statistic", value_name = "Stats Value")
stats_melt.loc[stats_melt["Stats Value"]==0, "Stats Value"] = np.nan
stats_melt

,Year Constructed,Statistic,Stats Value
0,1753-01-01,count,6.0
1,1802-01-01,count,2.0
2,1810-01-01,count,3.0
3,1812-01-01,count,1.0
4,1815-01-01,count,1.0
...,...,...,...
1419,2015-01-01,max,185525.0
1420,2016-01-01,max,184000.0
1421,2017-01-01,max,6720.0
1422,2018-01-01,max,12000.0


In [7]:
dropdown = alt.binding_select(options=["mean", "min", "25%", "50%", "75%", "max"], name="Statistic")
selection = alt.selection_point(fields=["Statistic"], bind=dropdown)
color = alt.condition(selection, alt.Color("Statistic:N"), alt.value("lightgray"))
opacity = alt.condition(selection, alt.value(1.0), alt.value(0.33))

linechart = alt.Chart(stats_melt).mark_line().encode(
    alt.Y("Stats Value:Q", scale=alt.Scale(type="log")),
    alt.X("Year Constructed:T"),
    color=color,
    opacity=opacity
).properties(
    height=200,
    width=600
).add_params(
    selection
)
linechart

alt.Chart(...)

In [8]:
myJekyllDir = '/Users/bradymb11/bradybrooks.github.io/assets/json/'
#linechart.save(myJekyllDir + 'altair_buildings.json')

In [9]:
stats = buildings.groupby("Usage Description")["Square Footage"].describe()
stats = stats[stats["count"] > 1]
stats

,count,mean,std,min,25%,50%,75%,max
Usage Description,,,,,,,,
Assembly,946.0,24734.226216,55910.393972,10.0,512.00,1343.0,22661.00,596000.0
Business,713.0,25543.049088,75818.400179,18.0,1512.00,4500.0,19534.00,1200000.0
Detention & Correc,729.0,12352.096022,20708.185841,25.0,420.00,6064.0,13800.00,307288.0
Education,257.0,52874.031128,72559.276608,100.0,4929.00,23096.0,72125.00,347981.0
Health Care,381.0,22072.580052,48636.584589,100.0,2909.00,3627.0,23800.00,671416.0
Industrial,1156.0,9171.032872,26170.259686,12.0,192.00,1300.0,5658.75,341100.0
Mercantile,76.0,4070.618421,5866.114296,32.0,856.25,1580.0,4245.00,31683.0
Residential,633.0,8240.756714,26948.789242,120.0,439.00,1452.0,3720.00,300097.0
Storage,2220.0,5121.590090,20418.389092,10.0,600.00,2400.0,5010.00,779732.0


In [10]:
stats_melt = stats.reset_index().melt("Usage Description", var_name="Statistic", value_name = "Stats Value")
stats_melt.loc[stats_melt["Stats Value"]==0, "Stats Value"] = np.nan
stats_melt = stats_melt[(stats_melt["Statistic"] != "count") & (stats_melt["Statistic"] != "std")]
stats_melt.head()

,Usage Description,Statistic,Stats Value
11,Assembly,mean,24734.226216
12,Business,mean,25543.049088
13,Detention & Correc,mean,12352.096022
14,Education,mean,52874.031128
15,Health Care,mean,22072.580052


In [11]:
dropdown = alt.binding_select(options=["mean", "min", "25%", "50%", "75%", "max"], name="Statistic")
selection = alt.selection_point(fields=["Statistic"], bind=dropdown, value="mean")
color = alt.condition(selection, alt.Color("Usage Description:N", legend=None), alt.value("lightgray"))
opacity = alt.condition(selection, alt.value(1.0), alt.value(0.25))

barchart = alt.Chart(stats_melt).mark_bar().encode(
    alt.Y("Stats Value:Q", title="Square Footage"),
    alt.X("Usage Description:N", title="Building Type"),
    color=color,
    opacity=opacity,
).properties(
    height=300,
    width=600
).add_params(
    selection
).transform_filter(
    selection
)
barchart

alt.Chart(...)

In [12]:
#barchart.save(myJekyllDir + 'altair_buildings2.json')